In [ ]:
import logging

import matplotlib.pyplot as plt
import numpy as np
from jaxtyping import ArrayLike
from molmass import Formula
from scipy.constants import mega

from atmodeller import debug_logger
from atmodeller.classes import EquilibriumModel
from atmodeller.containers import (
    ChemicalSpecies,
    FixedFugacityConstraint,
    SpeciesNetwork,
    ThermodynamicState,
)
from atmodeller.interfaces import FugacityConstraintProtocol, ThermodynamicStateProtocol
from atmodeller.output import Output
from atmodeller.solubility import get_solubility_models

logger: logging.Logger = debug_logger()
logger.setLevel(logging.INFO)

# Geological Outgassing

This notebook is available at `notebooks/geological_outgassing.ipynb` and is easiest to obtain by downloading the source code.

Atmodeller can perform "geological outgassing", for example, as appears in Gaillard and Scaillet (2014). The model below reproduces the basic result of Gaillard and Scaillet (2014), Figure 3.

In [ ]:
solubility_models = get_solubility_models()

# CO solubility neglected in Gaillard and Scaillet (low solubility in basaltic melt)
CO_g = ChemicalSpecies.create_gas("CO", solubility=solubility_models["CO_basalt_yoshioka19"])
O2_g = ChemicalSpecies.create_gas("O2")
CO2_g = ChemicalSpecies.create_gas("CO2", solubility=solubility_models["CO2_basalt_dixon95"])
H2_g = ChemicalSpecies.create_gas("H2", solubility=solubility_models["H2_basalt_hirschmann12"])
H2O_g = ChemicalSpecies.create_gas("H2O", solubility=solubility_models["H2O_basalt_mitchell17"])
# CH4 solubility neglected in Gaillard and Scaillet (low solubility in basaltic melt)
CH4_g = ChemicalSpecies.create_gas("CH4", solubility=solubility_models["CH4_basalt_ardia13"])
S2_g = ChemicalSpecies.create_gas("S2", solubility=solubility_models["S2_basalt_boulliung23"])
SO2_g = ChemicalSpecies.create_gas("SO2")
H2S_g = ChemicalSpecies.create_gas("H2S")
N2_g = ChemicalSpecies.create_gas("N2")

# Allow for graphite if stable, although the result demonstrates that graphite is not stable.
C_cr = ChemicalSpecies.create_condensed("C")

# Species lists for cases without nitrogen
species = SpeciesNetwork((CO_g, O2_g, CO2_g, H2_g, H2O_g, CH4_g, S2_g, SO2_g, H2S_g, C_cr))

model = EquilibriumModel(species)

Parameters from Gaillard and Scaillet (2014).

In [ ]:
temperature_celsius = 1300
# Note that the Figure 3 inset box gives -1.4 but the caption gives -1.5
dlog10fO2_QFM = -1.5

temperature_K = temperature_celsius + 273.15

# The shift is quoted relative to FMQ, so use the online buffer calculator to convert to get the
# absolute FMQ in log10 units at 1300 C (https://fo2.rses.anu.edu.au/fo2app/)
# log10fO2 below is at temperature = 1302.04081632653 C at 1 bar
log10fO2 = -7.27787591128208

# Account for the shift to get the absolute fO2 in log10 units
log10fO2 += dlog10fO2_QFM

Take the nominal concentrations of C, H, and O from Gaillard and Scaillet (2014) and convert to masses.

In [ ]:
# Initial volatile abundances (mole fraction) as estimated in undegassed Mid Ocean Ridge Basalts
CO2_initial = 600 / mega
H2O_initial = 1000 / mega
S_initial = 1000 / mega

# Approximate molar mass of basalt as pure SiO2, assuming some heavier and lighter components
# roughly cancel. A more accurate calculation could be performed if desired.
molar_mass_basalt = 60  # g/mol

# Mass fraction of volatiles per unit kg of basalt
C_mass = CO2_initial * Formula("C").mass / molar_mass_basalt
H_mass = H2O_initial * Formula("H2").mass / molar_mass_basalt
S_mass = S_initial * Formula("S").mass / molar_mass_basalt

logger.info("Mass of carbon = %0.5e", C_mass)
logger.info("Mass of hydrogen = %0.5e", H_mass)
logger.info("Mass of sulfur = %0.5e", S_mass)

In [ ]:
# Pressure range in bar, equivalent to Figure 3.
pressure_bar = np.logspace(-6, 3, num=100)
state: ThermodynamicStateProtocol = ThermodynamicState(temperature_K, pressure_bar)

# Impose the fO2 and volatile masses as constraints
fugacity_constraints: dict[str, FugacityConstraintProtocol] = {
    "O2_g": FixedFugacityConstraint(10**log10fO2)
}
mass_constraints: dict[str, ArrayLike] = {"C": C_mass, "H": H_mass, "S": S_mass}

model.solve(
    state=state,
    fugacity_constraints=fugacity_constraints,
    mass_constraints=mass_constraints,
    solver_type="basic",
)

output: Output = model.output
solution: dict[str, ArrayLike] = output.quick_look()

# Optionally, dump the data to Excel
# output.to_excel("gaillard")

Plot the gas speciation to compare with Gaillard and Scaillet (2014), Figure 3.

In [ ]:
# Get data as dataframes
data = output.to_dataframes()

# Plot total pressure on the x-axis
total_pressure = data["state"]["pressure"]

# Pick similar colours to Gaillard and Scaillet (2014) for easier visual comparison
colors = {
    "CO_g": "green",
    "CO2_g": "brown",
    "H2O_g": "darkblue",
    "H2_g": "blue",
    "O2S_g": "red",
    "H2S_g": "pink",
    "S2_g": "gold",
}

fig, ax = plt.subplots()

for species, color in colors.items():
    species_data = data[species]["volume_mixing_ratio"]
    ax.loglog(total_pressure, species_data, label=species, color=color)

ax.legend()
ax.set_title("Compare with Gaillard and Scaillet (2014), Figure 3")
ax.set_xlabel("Pressure (bar)")
ax.set_ylabel("Speciation in COHS gas")
ax.set_ylim(0.001, 1)